## FakeNewsNet

### Imports

In [ ]:
# python imports
import os.path
import urllib.request
import shutil
import zipfile
import json

# external library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# project imports


### Dataset

In [ ]:
def get_or_download_dataset():
    path = "./dataset/tweets"
    if not os.path.exists("./dataset"):
        print("downloading tweets...")
        os.mkdir("./dataset")
        url = "https://nusu-my.sharepoint.com/:u:/g/personal/e0809358_u_nus_edu/ETPkp1-0GbBHgB__wyeCS_QBE7_SFluzSCtocU0mUr3Jng"
        with urllib.request.urlopen(url) as response, open(path, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
            with zipfile.ZipFile(path + "/results.zip", "r") as zip_ref:
                zip_ref.extractall(path)
    return path

def to_dataframe(path, truncate_length = 10000):
    directories = os.listdir(path)
    json_all = []
    df_all = pd.DataFrame
    print("reading first " + str(truncate_length) + " json files in " + str(len(directories)) + " directories...")
    for index,directory in enumerate(directories):
        path_prefix = path + "/" + directory + "/tweets"
        files = os.listdir(path_prefix)
        # print("reading " + str(len(files)) + " json files from directory " + directory + " (" + str(index) + " of " + str(len(directories)) + ")")
        json_current = []
        for file in files:
            path_full = path_prefix + "/" + file
            with open(path_full, 'r') as json_file:
                json_current.append(json.loads(json_file.read()))
        df_current = pd.json_normalize(json_current)
        assert len(files) == len(json_current)
        assert len(files) == df_current.shape[0]
        json_all.extend(json_current)
        if len(json_all) >= truncate_length:
            break
        # df_all = pd.concat([df_all, df_current], axis=0, join='outer', sort=False)
    df_all = pd.json_normalize(json_all[:truncate_length])
    return df_all
    
def print_df(dfs):
    for df in dfs:
        print("==========")
        print(df.shape)
        # print(df.info())
    print("==========")
    
!pwd
!ls -l
path_dataset = get_or_download_dataset()
df_fake_1 = to_dataframe(path_dataset + "/gossipcop" + "/fake")
df_real_2 = to_dataframe(path_dataset + "/gossipcop" + "/real")
df_fake_2 = to_dataframe(path_dataset + "/politifact" + "/fake")
df_real_2 = to_dataframe(path_dataset + "/politifact" + "/real")
print_df([df_fake_1, df_real_2, df_fake_2, df_real_2])


### EDA

In [ ]:
def clean_dataset():
    pass